In [11]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [12]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [13]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Prueba 02")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/tarea-03/c4859157f91b4d25b330da4b251fa055



In [14]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [15]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [16]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [17]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 30,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy'
  }
  experiment.log_parameters(parameters)

In [18]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dense(30, activation = 'sigmoid')) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 30)                23550     
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [20]:
optimizer = RMSprop(learning_rate = 3.0)
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
5992/6000 [============================>.] - ETA: 0s - loss: 55.7665 - accuracy: 0.0991
Epoch 1: val_loss improved from inf to 85.40594, saving model to best_model.hdf5
6000/6000 [==============================] - 31s 5ms/step - loss: 55.7736 - accuracy: 0.0991 - val_loss: 85.4059 - val_accuracy: 0.0958
Epoch 2/30
  12/6000 [..............................] - ETA: 28s - loss: 55.2174 - accuracy: 0.1000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5999/6000 [============================>.] - ETA: 0s - loss: 55.7770 - accuracy: 0.0987
Epoch 2: val_loss improved from 85.40594 to 58.95736, saving model to best_model.hdf5
6000/6000 [==============================] - 27s 5ms/step - loss: 55.7742 - accuracy: 0.0987 - val_loss: 58.9574 - val_accuracy: 0.0974
Epoch 3/30
5988/6000 [============================>.] - ETA: 0s - loss: 55.6579 - accuracy: 0.1006
Epoch 3: val_loss improved from 58.95736 to 56.27194, saving model to best_model.hdf5
6000/6000 [==============================] - 28s 5ms/step - loss: 55.6538 - accuracy: 0.1007 - val_loss: 56.2719 - val_accuracy: 0.1010
Epoch 4/30
5993/6000 [============================>.] - ETA: 0s - loss: 55.8136 - accuracy: 0.0990
Epoch 4: val_loss improved from 56.27194 to 46.99394, saving model to best_model.hdf5
6000/6000 [==============================] - 26s 4ms/step - loss: 55.8158 - accuracy: 0.0990 - val_loss: 46.9939 - val_accuracy: 0.0892
Epoch 5/30
5991/6000 [==========================

In [21]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/c4859157f91b4d25b330da4b251fa055
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.09771666675806046, 0.10278332978487015)
COMET INFO:     batch_accuracy [18000]         : (0.0, 0.4000000059604645)
COMET INFO:     batch_loss [18000]             : (2.5121889114379883, 93.85111236572266)
COMET INFO:     epoch_duration [30]            : (26.27716798000006, 30.845775239999966)
COMET INFO:     loss [30]                      : (55.5382194519043, 55.85319137573242)
COMET INFO:     val_accuracy [30]              : (0.08919999748468399, 0.10279999673366547)
COMET INFO:   